Goal: Use the 2 Photon Imaging module to store data from TiltBrush rendering of neural anatomy, with length/width/volume/center of mass measurements for each dendtritic spine.

Assumption 1: the manifold, "np.ones((5, 5, 3))', should describe the coordinates of the points/triangles in our image segments.

Assumption 2: ROIs represent the vertices we use to define dendtritic spine length and width.

Assumption 3: Image segmentation should be done for dendrite segments

In [1]:
import os
import h5py
#from tqdm import tqdm #Loading bar wrapper
import numpy as np
import open3d as o3d
import csv
from pynwb import NWBFile, NWBHDF5IO
from datetime import datetime
from dateutil.tz import tzlocal
from pynwb import NWBFile
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence
from pynwb.device import Device
#from nwbwidgets import nwb2widget



Read in renderings from the different segments of our image analysis

In [2]:
os.chdir('C:/Users/Mrika/OneDrive/TanLab/NWBHackathonFiles/HackthonFiles/ObjectModels')

soma = o3d.io.read_triangle_mesh("soma.obj")
dendrites = o3d.io.read_triangle_mesh("dendrites.obj")
spines = o3d.io.read_triangle_mesh("spines.obj")
mushrooms = o3d.io.read_triangle_mesh("mushrooms.obj")

#Different ways of manipulating the soma data
print(soma)
soma_triangles = np.asarray(soma.triangles)
print(soma_triangles.shape)


geometry::TriangleMesh with 756 points and 252 triangles.
(252, 3)


Please see SourceDataforSoma.txt in the raw data folder for complete description of data from soma.obj

In [3]:
#Display Soma shape in another window
soma.compute_vertex_normals()
o3d.visualization.draw_geometries([soma])

We are planning to write an add in for Blender to extract ROI data from vertex objects created in each dendtritic spine object.  For the purposes of the hackathon, we will hard-code the test variables.

In [4]:
ROI_ID = 1
L1 = [1.1, 1.2, 3.1,1] # x1, y1, z1, weight1
L2 = [1.2, 1.4,1.8,1]
W1 = [2.2, 4.1, 3.6, 1]
W2 = [3.2, 4.4, 3.2, 1]

#Other data associated with spines that doesn't belong in ROI table.  Probably belongs in columns of the image segmentation. :
volume = 12
center_of_mass = [1.2, 1.4,1.8]



In [5]:
#Code from the calcium imaging tutorial

nwbfile = NWBFile('my first synthetic recording', 'EXAMPLE_ID', datetime.now(tzlocal()),
                  experimenter='Dr. Bilbo Baggins',
                  lab='Bag End Laboratory',
                  institution='University of Middle Earth at the Shire',
                  experiment_description=('I went on an adventure with thirteen '
                                          'dwarves to reclaim vast treasures.'),
                  session_id='LONELYMTN')

#Create metadata about our microscope
device = Device('Our microscope')
nwbfile.add_device(device)
optical_channel = OpticalChannel('my_optchan', 'description', 500.)

#QUESTION: HOW DO WE CREATE THE MANIFOLD?

imaging_plane = nwbfile.create_imaging_plane('my_imgpln', optical_channel, 'a very interesting part of the brain',
                                             device, 600., 300., 'GFP', 'my favorite brain location',
                                             np.ones((5, 5, 3)), 4.0, 'manifold unit', 'A frame to refer to')

#Add datasets  to the processing module
mod = nwbfile.create_processing_module('RenderedNeuron', 'Contains triangle mesh data from a VR neuron')



#This links our processed data to the original image/images
image_series = TwoPhotonSeries(name='test_iS', dimension=[2],
                               external_file=['images.tiff'], imaging_plane=imaging_plane,
                               starting_frame=[0], format='tiff', starting_time=0.0, rate=1.0)
nwbfile.add_acquisition(image_series)

img_seg = ImageSegmentation()
mod.add(img_seg)
ps = img_seg.create_plane_segmentation('output from segmenting my favorite imaging plane',imaging_plane, 'my_planeseg', image_series)

#Example pixel and voxel masks

w, h = 3, 3
pix_mask1 = [(0, 0, 1.1), (1, 1, 1.2), (2, 2, 1.3)]
img_mask1 = [[0.0 for x in range(w)] for y in range(h)]
img_mask1[0][0] = 1.1
img_mask1[1][1] = 1.2
img_mask1[2][2] = 1.3
ps.add_roi(pixel_mask=pix_mask1, image_mask=img_mask1)

pix_mask2 = [(0, 0, 2.1), (1, 1, 2.2)]
img_mask2 = [[0.0 for x in range(w)] for y in range(h)]
img_mask2[0][0] = 2.1
img_mask2[1][1] = 2.2
ps.add_roi(pixel_mask=pix_mask2, image_mask=img_mask2)

In [6]:
#save file
with NWBHDF5IO('prototype.nwb', 'w') as io:
    io.write(nwbfile)
    